In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import math
import glob
import pickle
import statistics
import scipy.stats as stats
from sklearn.neighbors import KernelDensity
import dask
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [2]:
def get_files():
    models = glob.glob("/terra/data/cmip5/global/historical/*")
    avail={}
    for model in models:
        zg = glob.glob(str(model)+"/r1i1p1/day/2deg/zg*")
        try:
            test = zg[0]
            avail[model.split('/')[-1]] = zg
        except:
             pass
    return avail

In [3]:
files = get_files()
files['NOAA'] = glob.glob("/home/pmarsh/NOAA_2deg/NOAA_zg/*.nc") #need 2degs!
files['ERA5'] = glob.glob("/home/pmarsh/NOAA_2deg/ERA5_zg/*.nc")
files.pop('MIROC-ESM')

['/terra/data/cmip5/global/historical/MIROC-ESM/r1i1p1/day/2deg/zg_day_MIROC-ESM_historical_r1i1p1_19500101-20051231.nc']

In [4]:
def contourise(x):
    x = x.fillna(0)
    x = x.where((x>=limit))
    x = x/x
    return x

In [13]:
results={}
for model in files:
    print(model)
    x = xr.open_mfdataset(files[model])
    if model == 'NOAA':
        x = x.rename({'hgt':'zg'})
        x = x.rename({'level':'plev'})
        x = x.sel(plev=850)
        x = x.sel(time=slice('1950','2005'))
    elif model == 'ERA5':
        x = x.rename({'level':'plev'})
        x = x.sel(plev=850)
        x = x.sel(time=slice('1979','2005'))
    else:
        x = x.sel(plev=85000)
        x = x.sel(time=slice('1950','2005'))
        x = x.load()
    x = x.sel(lat=slice(-60,0))
    x = x[['zg']]
    x = x.assign_coords(lon=(((x.lon + 180) % 360) - 180))
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        x = x.sortby(x.lon)
    x = x.sel(lon=slice(-50,20))
    x = x.resample(time="QS-DEC").mean(dim="time",skipna=True)
    x = x.load()
    limit = np.nanquantile(x.zg.values,0.9)
    results[model]={}
    for seas in ['DJF','MAM','JJA','SON']:
        mean_seas = x.where(x.time.dt.season==str(seas)).dropna(dim='time')
        mean_seas = contourise(mean_seas).zg.fillna(0).mean(dim='time')
        results[model][seas] = mean_seas.fillna(0)
    x.close()

IPSL-CM5A-LR
MPI-ESM-MR
MRI-ESM1
MIROC-ESM-CHEM
EC-EARTH
HadGEM2-CC
bcc-csm1-1-m
FGOALS-g2
ACCESS1-0
GFDL-CM3
CanESM2
IPSL-CM5B-LR
MPI-ESM-LR
CMCC-CM
GFDL-ESM2G
MIROC5
MPI-ESM-P


/share/apps/anaconda3/envs/dask/lib/python3.8/site-packages/xarray/coding/times.py:113: SerializationWarning: Ambiguous reference date string: 850-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


CMCC-CMS
CNRM-CM5
IPSL-CM5A-MR
ACCESS1-3
MRI-CGCM3
bcc-csm1-1
CMCC-CESM
HadGEM2-AO
BNU-ESM
NorESM1-M
HadCM3
GFDL-ESM2M
NOAA


/share/apps/anaconda3/envs/dask/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


ERA5


In [14]:
scores=[]
for index in results:
    reference = index
    for index in results:
        MAE=[]
        for season in ['DJF','MAM','JJA','SON']:
            ref = results[reference][season]
            x = results[index][season]
            for i in ref.lat.values:
                for j in ref.lon.values:
                    MAE.append(float(np.abs(ref.sel(lat=i).sel(lon=j) - x.sel(lat=i).sel(lon=j)).values))
        scores.append([reference,index,np.mean(MAE)])

In [15]:
models = list(results.keys())

In [16]:
df = pd.DataFrame(models,columns = ['models'])
start = 0
end = len(models)
for index in models:
    df[index] = np.array(scores[start:end])[:,-1].astype(float)
    start = start + len(models)
    end = end + len(models)

In [17]:
df = df.set_index('models')

In [18]:
df.to_csv('High_2D_Linkage.csv')

In [19]:
df = df.dropna(0,how='all').dropna(1,how='all')

In [20]:
df

,IPSL-CM5A-LR,MPI-ESM-MR,MRI-ESM1,MIROC-ESM-CHEM,EC-EARTH,HadGEM2-CC,bcc-csm1-1-m,FGOALS-g2,ACCESS1-0,GFDL-CM3,...,MRI-CGCM3,bcc-csm1-1,CMCC-CESM,HadGEM2-AO,BNU-ESM,NorESM1-M,HadCM3,GFDL-ESM2M,NOAA,ERA5
models,,,,,,,,,,,,,,,,,,,,,
IPSL-CM5A-LR,0.000000,0.050070,0.100280,0.028047,0.061090,0.059945,0.038967,0.049073,0.057044,0.051882,...,0.100280,0.048451,0.034470,0.072474,0.028928,0.048275,0.038746,0.048720,0.059597,0.058738
MPI-ESM-MR,0.050070,0.000000,0.100246,0.037256,0.023086,0.035778,0.041781,0.047648,0.043296,0.057555,...,0.100246,0.030544,0.032677,0.057353,0.038650,0.060425,0.037639,0.033258,0.039119,0.033989
MRI-ESM1,0.100280,0.100246,0.000000,0.100242,0.100203,0.100308,0.100151,0.100178,0.101313,0.065751,...,0.000000,0.100111,0.100235,0.101110,0.100211,0.100080,0.100136,0.100107,0.105688,0.100285
MIROC-ESM-CHEM,0.028047,0.037256,0.100242,0.000000,0.044411,0.045001,0.022866,0.040323,0.043417,0.049635,...,0.100242,0.033390,0.026492,0.069628,0.021774,0.056300,0.032676,0.041932,0.046504,0.041648
EC-EARTH,0.061090,0.023086,0.100203,0.044411,0.000000,0.035985,0.046032,0.050908,0.045330,0.062180,...,0.100203,0.032167,0.040897,0.059644,0.043928,0.063454,0.044410,0.033988,0.041001,0.034533
HadGEM2-CC,0.059945,0.035778,0.100308,0.045001,0.035985,0.000000,0.040548,0.043155,0.020760,0.075962,...,0.100308,0.025268,0.050128,0.045342,0.039196,0.073990,0.045832,0.036295,0.024378,0.016961
bcc-csm1-1-m,0.038967,0.041781,0.100151,0.022866,0.046032,0.040548,0.000000,0.034620,0.041092,0.055562,...,0.100151,0.027457,0.025843,0.064846,0.026244,0.059247,0.030499,0.043598,0.042017,0.038046
FGOALS-g2,0.049073,0.047648,0.100178,0.040323,0.050908,0.043155,0.034620,0.000000,0.044148,0.065715,...,0.100178,0.028992,0.040385,0.071356,0.036972,0.073586,0.044062,0.047656,0.053441,0.044173
ACCESS1-0,0.057044,0.043296,0.101313,0.043417,0.045330,0.020760,0.041092,0.044148,0.000000,0.073972,...,0.101313,0.032250,0.050212,0.050719,0.038529,0.074419,0.045205,0.044068,0.031248,0.029488
